# Surface Code Encoding

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from qiskit import QuantumCircuit, execute, QuantumRegister, ClassicalRegister, Aer
import matplotlib.pyplot as plt

## Stabilizer Eigenstates

Following Fowler et. al [1], we see that the two-qubit operators $Z_a Z_b$ commute with $X_a X_b$. The eigenstates of these *stabilizers* are the Bell states (**Table II**):

<p align="center">
<img width="245" alt="stabilizer-eigenstates" src="https://user-images.githubusercontent.com/293681/88832601-0f9a5800-d19f-11ea-88fe-a8191e21a67e.png">
</p>

[1] Fowler, A. G., Mariantoni, M., Martinis, J. M. & Cleland, A. N. Surface codes: Towards practical large-scale quantum computation. Phys. Rev. A 86, 032324 (2012).


In [ ]:
qr = QuantumRegister(2, name='q')
an = QuantumRegister(1, name='ancilla')

rounds = 2
measurements = [ClassicalRegister(2, name="c{}".format(i+1)) for i in range(rounds)]

circ = QuantumCircuit(qr, an, *measurements)

def ZZ_meas(round):
    circ.reset(an)
    circ.cx(0, 2)
    circ.cx(1, 2)
    circ.measure(an, measurements[round][1])
    circ.barrier()

def XX_meas(round):
    circ.reset(an)
    circ.h(an)
    circ.cx(an, 0)
    circ.cx(an, 1)
    circ.h(an)
    circ.measure(an, measurements[round][0])
    circ.barrier()

for i in range(rounds):
    ZZ_meas(i)
    XX_meas(i)

circ.draw(output='mpl', fold=35)

Running this circuit on the four Bell states, and the stabilizer is stable:

In [ ]:
import itertools
for q0_gate, q1_gate in itertools.product(["i", "x"], repeat=2):  
    test_circ = QuantumCircuit(qr, an, *measurements)
    
    # Set up our Bell "generator" states
    getattr(test_circ, q0_gate)(0) #i.e. test_circ.i(0) or test_circ.x(0)
    getattr(test_circ, q1_gate)(1) #i.e. test_circ.i(1) or test_circ.x(1)
    
    # Create Bell state
    test_circ.h(0)
    test_circ.cx(0, 1)
    test_circ.barrier()

    test_circ = test_circ.combine(circ)
    print("Bell state %s%s:" % (q0_gate, q1_gate), execute(test_circ, Aer.get_backend('qasm_simulator'), shots=1).result().get_counts())
# Final Circuit Example
test_circ.draw(output='mpl', fold=35)

Even though our projective measurement is only on the ancilla qubit, all of our qubits are entangled through the stabilizer measurement process. Thus, the ancilla measurement *does* induce back-action on our actual qubit. For example, let's run the circuit on the $|00\rangle$ state, which is **not** a mutual eigenstate of the stabilizers:

In [ ]:
execute(circ, Aer.get_backend('qasm_simulator')).result().get_counts()

Roughly half of the time we get `00 00`, and the other half `01 01`. If you go through the circuit, it's straightforward to verify that the first $XX$ measurement will project $|00\rangle$ into either the $|00\rangle$ + $|11\rangle$ or $|00\rangle$ - $|11\rangle$ Bell state (and their respective stabilizer eigenvalues).

In the absence of errors, subsequent measurements of the eigenvalue after the first stabilizer measurement will remain the same! This is known as the *quiescent state* of the code, and is an important part of the initialization process for surface codes.

## Toric Code to Surface Code
Korotkov’s law:  “In quantum mechanics you don’t see what you get, you get what you see.”

See James Wooten's [Decodoku toric code posts](http://decodoku.blogspot.com/2016/03/6-toric-code.html) for a great introduction to the intuition.

<p align="center">
<img width="800" alt="toric-code" src="https://user-images.githubusercontent.com/293681/98292863-6c19ca00-1f7b-11eb-85d5-c52ea6518e50.png">
</p>

## d=3 Surface Code with 17 Qubits

Following [2], we can generalize this into a d=3 rotated surface code with 9 data qubits and 8 ancillas. Please see the reference for details about how the circuit is constructed, including a subtlety with the direction of stabilzer measurements in each cell.

<p align="center">
<img width="800" alt="syndrome-measurement" src="https://user-images.githubusercontent.com/293681/98292541-dc741b80-1f7a-11eb-9e37-19333f821ee0.png">
<img width="400" alt="d-3-code-lattice" src="https://user-images.githubusercontent.com/293681/98292642-04637f00-1f7b-11eb-9924-8a02f512d65c.jpeg">
</p>

[2] Tomita, Y. & Svore, K. M. Low-distance surface codes under realistic quantum noise. Phys. Rev. A 90, 062320 (2014).

In [ ]:
# [circ.x(i) for i in data]
def stabilize(circ, i):
    # Top left
    circ.h(mx[0])
    circ.cx(mx[0], data[1])
    circ.cx(mx[0], data[0])
    circ.h(mx[0])
    circ.cx(data[1], mz[0])
    circ.cx(data[4], mz[0])
    circ.cx(data[3], mz[0])
    circ.cx(data[0], mz[0])
    circ.barrier()
    
    # Top right
    circ.h(mx[1])
    circ.cx(mx[1], data[2])
    circ.cx(mx[1], data[1])
    circ.cx(mx[1], data[5])
    circ.cx(mx[1], data[4])
    circ.h(mx[1])
    circ.cx(data[2],mz[1])
    circ.cx(data[5],mz[1])
    circ.barrier()
    
    # Bottom left
    circ.cx(data[3], mz[2])
    circ.cx(data[6], mz[2])
    circ.h(mx[2])
    circ.cx(mx[2], data[4])
    circ.cx(mx[2], data[3])
    circ.cx(mx[2], data[7])
    circ.cx(mx[2], data[6])
    circ.h(mx[2])
    circ.barrier()

    # Bottom right
    circ.h(mx[3])
    circ.cx(mx[3], data[8])
    circ.cx(mx[3], data[7])
    circ.h(mx[3])
    circ.cx(data[5], mz[3])
    circ.cx(data[8], mz[3])
    circ.cx(data[4], mz[3])
    circ.cx(data[7], mz[3])
    circ.barrier()
    
    circ.measure(mz, measurements[i][0:4])
    circ.measure(mx, measurements[i][4:8])
    circ.reset(mz)
    circ.reset(mx)
    circ.barrier()

In [ ]:
data = QuantumRegister(9, name='data')
mx = QuantumRegister(4, name='mx')
mz = QuantumRegister(4, name='mz')

rounds=1  # The actual number of rounds will always be more than 1, since the first round creates the quiescent state
measurements = [ClassicalRegister(8, name="c{}".format(i+1)) for i in range(rounds + 1)]
logical = QuantumRegister(1, name="logical")

base_circ = QuantumCircuit(data, mz, mx, *measurements, logical)

def get_stabilized_circ(base_circuit, rounds):
    circ = base_circuit.copy()
    for i in range(rounds + 1):
        stabilize(circ, i)
    return circ

In [ ]:
circ = get_stabilized_circ(base_circ, rounds)
circ.draw(output='mpl', fold=150)

### "Circuit generator" from our library

Works for arbitrary d (still odd, of course)! We basically iterate over each "syndrome face" and run the X or Z entanglement. Take a look at the code for more details, including the messy calculations for the lattice locations.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.insert(0, "../")
sys.path.insert(0, "../../")
from topological_codes import XXZZQubit

In [ ]:
qubit = XXZZQubit({'d':3})
# print(qubit)
qubit.stabilize()
qubit.stabilize()
qubit.draw(output='mpl', fold=50)

Just like before, the $XXXX$ measurements will project our starting state into a quiescent state. Similarly, the $ZZZZ$ measurement doesn't do anything here, since both the starting state and the quiescent states are already Z-eigenstates.

We make four different $XXXX$ measurements, and each has two eigenvalues, so in this case there are $2^4 = 16$ possible combinations of syndrome measurements.

We can run this on the simulator and confirm. We see 3 groups of 8 bits, which are the syndrome measurement rounds. These are stable, and the first 4 bits permute through the 16 possible $XXXX$ syndromes. The output format is:

$$\underbrace{X_3X_2X_1X_0Z_3Z_2Z_1Z_0}_\text{Round 1} \quad \underbrace{X_3X_2X_1X_0Z_3Z_2Z_1Z_0}_\text{Round 0}$$

In [ ]:
execute(qubit.circ, Aer.get_backend('qasm_simulator')).result().get_counts()

From Decodoku posts by James Wooten:

http://decodoku.blogspot.com/2016/03/6-toric-code-part-2.html

<p align="center">
</p>

In our circuit, the Z-operator can be picked as any row of data qubits. We choose the top row:

In [ ]:
qubit.readout_z()
qubit.draw(output='mpl', fold=150)

With the readout, the output format is:
$$\underbrace{L}_\text{Z Readout}~\underbrace{X_3X_2X_1X_0Z_3Z_2Z_1Z_0}_\text{Round 1} \quad \underbrace{X_3X_2X_1X_0Z_3Z_2Z_1Z_0}_\text{Round 0}$$

In [ ]:
execute(qubit.circ, Aer.get_backend('qasm_simulator')).result().get_counts()

Applying the X operator, we can get a $|1_L\rangle$ state:

In [ ]:
qubit = XXZZQubit({'d':3})
qubit.stabilize()
qubit.stabilize()
qubit.logical_x()
qubit.stabilize()
qubit.readout_z()

execute(qubit.circ, Aer.get_backend('qasm_simulator')).result().get_counts()

If we initialize the circuit with $|+\rangle$ data qubits, we get a logical $|+\rangle$ state:

In [ ]:
qubit = XXZZQubit({'d':3})
qubit.logical_x_plus_reset()
qubit.stabilize()
qubit.stabilize()
qubit.readout_x()  # Note that we need to readout X now instead of Z
qubit.draw(output='mpl', fold=150)

In [ ]:
execute(qubit.circ, Aer.get_backend('qasm_simulator')).result().get_counts()

The logical-Z operator will flip this to $|-\rangle$

In [ ]:
qubit = XXZZQubit({'d':3})
qubit.logical_x_plus_reset()
qubit.stabilize()

qubit.logical_z()

qubit.stabilize()
qubit.readout_x() 

execute(qubit.circ, Aer.get_backend('qasm_simulator')).result().get_counts()

### For fun, the d=5 surface code

In [ ]:
qubit = XXZZQubit({'d':5})
qubit.stabilize()
qubit.draw(output='mpl', fold=170)

We can test the effect of errors. We insert X-flips at qubits 6 and 9. The output syndrome has the format:

$$\underbrace{L}_\text{Z Readout}~\underbrace{X_NX_{N-1}...X_0Z_NZ_{N-1}...Z_0}_\text{Round 1} \quad \underbrace{X_NX_{N-1}...X_0Z_NZ_{N-1}...Z_0}_\text{Round 0}$$

We should get syndrome flips at $Z_0, Z_2, Z_4, Z_5$:
<p align="center">
<img width="500" alt="torus" src="https://user-images.githubusercontent.com/293681/98292078-1a247480-1f7a-11eb-90f3-822a55f1a8ff.jpg">
</p>

In [ ]:
qubit = XXZZQubit({'d':3})
qubit.stabilize()
qubit.circ.x(3)
# qubit.circ.x(6)
# qubit.circ.z(0)
# qubit.circ.x(9)
qubit.stabilize()
qubit.readout_z()

In [ ]:
results = execute(qubit.circ, Aer.get_backend('qasm_simulator'), shots=1).result().get_counts()
readout_string = list(results.keys())[0]
print(readout_string)
qubit.parse_readout(readout_string)

### Using lattice readout

We can also readout the lattice (i.e. measure each data qubit) along a specified axis to get BOTH a final syndrome measurement and a logical readout. 

Suppose we are trying to make a logical Z readout. In this case, we will measure each data qubit along the Z axis. This will allow us to extract a logical Z readout and a final set of Z syndrome measurements. 

However, we will not be able to extract a final set of X syndrome measurements. This is ok. When measuring logical Z readout, we only care about X flips on any data qubits, which are fully detected by the Z syndrome graph.

In [ ]:
qubit = XXZZQubit({'d':5})
qubit.stabilize()
qubit.circ.x(0)
qubit.circ.z(0)
qubit.stabilize()
qubit.circ.x(0)
qubit.circ.z(0) #this will not be detected, but that's ok!
qubit.lattice_readout_z()

In [ ]:
logical_readout = 0
while logical_readout != 1:
    results = execute(qubit.circ, Aer.get_backend('qasm_simulator'), shots=1).result().get_counts()
    readout_string = list(results.keys())[0]
    print("Readout String: ", readout_string)
    logical_readout, syndromes = qubit.parse_readout(readout_string, "Z")
    print("Logical Readout: ", logical_readout)
    print("Syndromes: \n", syndromes)

First, notice that the first chunk of the readout string represents the result of a readout of all 25 data qu
As shown above, the 'Z' syndrome graph detects errors from both rounds, while the 'X' syndrome graph detects errors from just the first round as it does not pick up on the second Z data flip. Again, this is ok, since we are performing a logical Z readout!

Notice that the logical Z readout is affected by the x flip on the 0th data qubit. Fear not! This will be corrected by MWPM decoding. 